In [8]:
import pandas as pd
import os
import numpy as np

In [12]:
folder = "./data"

files = os.listdir(folder)

datasets = {file_name.replace(".xlsx", ""): pd.read_excel(os.path.join(folder, file_name)) for file_name in files}

for name, df in datasets.items():
    print(f"\n{name} loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.apply(lambda col: col.unique()))


EOAL_PAIS_DL loaded: 943 rows, 7 columns
Unnamed: 0          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
AÑO                                          [2022, 2023, 2024, 2025]
MES                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
PAIS                [Alemania, Austria, Bélgica, Dinamarca, España...
LUGAR_RESIDENCIA    [Residentes en el Extranjero, Residentes en Es...
VIAJEROS            [2892, 0, 717, 39580, 3798, 3592, 2898, 8086, ...
PERNOCTACIONES      [6133, 0, 1603, 86870, 8338, 8065, 6587, 17311...
dtype: object

CST_GASTO_CONSUMO_TURISTICO_DL loaded: 112 rows, 9 columns
Unnamed: 0                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
AÑO                                          [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
PRODUCTO                                    [Agencias de viajes y otros servicios de reser...
NUM_GASTO_INTERIOR                          [6910.6, 23393.6, 28999.0, 86710.1, 5383.0, 77...
NUM_GASTO_INTERNO 

In [ ]:
# ¿Están los turistas dispuestos a pagar más por opciones sostenibles y auténticas?

In [32]:
datasets["EGATUR_DL"].columns.tolist()

['AÑO',
 'MES',
 'PAIS_RESIDENCIA',
 'TIPO_VISITANTE',
 'CCAA_DESTINO',
 'PERNOCTACIONES',
 'GASTO_TOTAL']

In [28]:
excursionistas = datasets["EGATUR_DL"]["TIPO_VISITANTE"].isin(["Excursionista no residente (no tránsito)", "Excursionista no residente en tránsito"]).sum()

np.int64(4437)

In [33]:
evolucion = datasets["EGATUR_DL"].groupby(["AÑO", "TIPO_VISITANTE"]).size().reset_index(name="TOTAL")

In [36]:
egatur_dl = datasets["EGATUR_DL"].copy()

egatur_dl["TIPO_VISITANTE_GROUP"] = egatur_dl["TIPO_VISITANTE"].apply(lambda x: "Turista" if "Turista" in x else "Excursionista")

In [56]:
evolucion = egatur_dl.groupby(["AÑO", "TIPO_VISITANTE_GROUP"]).size().reset_index(name="TOTAL")
visitantes_ccaa = (egatur_dl.groupby(["AÑO", "CCAA_DESTINO", "TIPO_VISITANTE_GROUP"]).size().reset_index(name="TOTAL"))

In [53]:
evolucion

,AÑO,TIPO_VISITANTE_GROUP,TOTAL
0,2018,Excursionista,1077
1,2018,Turista,3613
2,2019,Excursionista,1140
3,2019,Turista,3612
4,2020,Excursionista,597
5,2020,Turista,2474
6,2021,Excursionista,730
7,2021,Turista,3209
8,2022,Excursionista,893
9,2022,Turista,3478


In [54]:
evolucion_total_anual

,AÑO,TOTAL
0,2018,4690
1,2019,4752
2,2020,3071
3,2021,3939
4,2022,4371
5,2023,3672
6,2024,3489
7,2025,1430


In [85]:
ccaa_tradicionales = ["Cataluña", "Andalucía", "Comunidad Valenciana", "Illes Balears", "Madrid", "Canarias"]
ccaa_alternativas = ["Galicia", "Cantabria", "Aragón", "Extremadura", "Castilla y León", "La Rioja", "Asturias", "Navarra", "País Vasco"]

def clasificar_ccaa(ccaa):
    if ccaa in ccaa_tradicionales:
        return "Tradicional"
    elif ccaa in ccaa_alternativas:
        return "Emergente"
    else:
        return "Otra"

visitantes_ccaa["TIPO_CCAA"] = visitantes_ccaa["CCAA_DESTINO"].apply(clasificar_ccaa)

visitantes_ccaa.to_csv("./exports/visitantes_ccaa.csv", index=False)
visitantes_ccaa.sort_values(by="TOTAL", ascending=False)

,AÑO,CCAA_DESTINO,TIPO_VISITANTE_GROUP,TOTAL,TIPO_CCAA
13,2018,Cataluña,Turista,325,Tradicional
51,2019,Cataluña,Turista,324,Tradicional
205,2024,Cataluña,Turista,323,Tradicional
186,2023,Cataluña,Turista,320,Tradicional
39,2019,Andalucía,Turista,318,Tradicional
...,...,...,...,...,...
29,2018,Melilla,Excursionista,1,Otra
156,2022,Castilla-La Mancha,Excursionista,1,Otra
141,2021,Principado de Asturias,Excursionista,1,Otra
119,2021,Castilla-La Mancha,Excursionista,1,Otra


In [61]:
totales_por_ccaa = visitantes_ccaa.groupby("CCAA_DESTINO")["TOTAL"].sum().reset_index()
totales_por_ccaa.sort_values(by="TOTAL", ascending=False)

,CCAA_DESTINO,TOTAL
6,Cataluña,2948
0,Andalucía,2700
9,Comunidad de Madrid,2427
10,Comunitat Valenciana,2311
16,País Vasco,2231
4,Castilla y León,1961
13,Illes Balears,1936
12,Galicia,1802
2,Canarias,1703
1,Aragón,1399


In [74]:
visitantes_ccaa_pivot = visitantes_ccaa.pivot_table(index=["AÑO", "CCAA_DESTINO"], columns="TIPO_VISITANTE_GROUP", values="TOTAL", fill_value=0).reset_index()
visitantes_ccaa_pivot["TOTAL"] = (visitantes_ccaa_pivot["Turista"] + visitantes_ccaa_pivot["Excursionista"])

visitantes_ccaa_pivot.sort_values(by=["AÑO", "TOTAL"], ascending=[True, False], inplace=True)

In [75]:
visitantes_ccaa_pivot

TIPO_VISITANTE_GROUP,AÑO,CCAA_DESTINO,Excursionista,Turista,TOTAL
6,2018,Cataluña,164.0,325.0,489.0
0,2018,Andalucía,149.0,314.0,463.0
9,2018,Comunidad de Madrid,64.0,315.0,379.0
10,2018,Comunitat Valenciana,67.0,309.0,376.0
16,2018,País Vasco,84.0,280.0,364.0
...,...,...,...,...,...
145,2025,Comunidad Foral de Navarra,0.0,52.0,52.0
154,2025,Principado de Asturias,0.0,51.0,51.0
148,2025,Extremadura,0.0,42.0,42.0
151,2025,La Rioja,0.0,28.0,28.0


In [76]:
visitantes_ccaa_pivot.to_csv("./exports/visitantes_ccaa_pivot.csv", index=False)